In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/HW4'

/content/drive/MyDrive/Colab Notebooks/HW4


In [9]:
# imports

import os
from pathlib import Path
from omegaconf import OmegaConf

import torch

from src.data_pipeline import CIFAR10Pipeline
from src.classifier import Classifier
from src.runner import run_experiment, compare_experiments, build_model_path

In [10]:
# load config

cfg = OmegaConf.load("conf/config.yaml")

print("="*30)
print("Setting up data pipeline...")
print("="*30)

data_pipeline = CIFAR10Pipeline(cfg)
train_loader, val_loader, test_loader = data_pipeline.setup()

print(f"\nDataset: CIFAR-10")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")
print(f"Batch size: {cfg.train.batch_size}\n")

Setting up data pipeline...

Dataset: CIFAR-10
Training samples: 45000
Validation samples: 5000
Test samples: 10000
Batch size: 128



In [11]:
def print_exp_params(cfg):
  model_name = cfg.model.model_name
  print("="*30)
  print(f"Experiment Configuration")
  print("="*30)
  print(f"  Mode: {cfg.mode}")
  print(f"  Model: {cfg.model.model_name}")
  print(f"  Optimizer: {cfg.train.optimizer}")
  print(f"  Learning Rate: {cfg.train.learning_rate}")
  print(f"  Batch Size: {cfg.train.batch_size}")
  print(f"  Epochs: {cfg.train.num_epochs}")
  print(f"  Weight Decay: {cfg.train.weight_decay}")
  if model_name.lower() == 'customcnn':
      print(f"  CNN Layers: {cfg.model.cnn.num_layers}")
      print(f"  Use BN: {cfg.model.cnn.use_batch_norm}")
      print(f"  Use Dropout: {cfg.model.cnn.use_dropout}")
      if cfg.model.cnn.use_dropout:
        print(f"  Dropout Rate: {cfg.model.cnn.dropout_rate}")
      print(f"  FC Layers: {cfg.model.cnn.fc_layers}")

In [12]:
# initialize experiment params

cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

# Dictionary to store results from all experiments
experiment_results = {}

# Run experiment for the configured model
print_exp_params(cfg)


Experiment Configuration
  Mode: pretrained
  Model: ResNet18
  Optimizer: adam
  Learning Rate: 0.001
  Batch Size: 128
  Epochs: 1
  Weight Decay: 0.0001


In [13]:
# run single experiment

mode = cfg.mode
model_name = cfg.model.model_name

results = run_experiment(
    cfg=cfg,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=cifar10_classes,
    mode=mode
)

experiment_results[model_name] = results

print("\n" + "="*30)
print("Experiment completed successfully!")
print("="*30)


Initializing ResNet18...
Mode: pretrained (freeze=off)
Built ResNet18 pretrained with 11,181,642 total parameters
  Trainable parameters: 11,181,642

Training ResNet18_pretrained_adam for 1 epochs...


Epoch [1/1] Train Loss: 0.9104 Acc: 69.10% | Val Loss: 0.7485 Acc: 74.52%
Model saved to checkpoints/ResNet18_pretrained_adam_weights/weights.pth
Training curves saved to results/ResNet18_pretrained_adam/training_curves.html


TypeError: evaluate_model() got an unexpected keyword argument 'cfg'

In [ ]:
# Load and evaluate existing model weights

# Example 1: Load weights using config
# Set cfg.load_weights = True and cfg.mode to match the model you want to load
cfg.load_weights = True
cfg.mode = 'pretrained'  # or 'scratch'
cfg.train.freeze = False  # If loading pretrained model, set freeze accordingly

# Check what model path will be used
model_path = build_model_path(cfg)
print(f"Model path: {model_path}")
print(f"Model exists: {Path(model_path).exists()}")

# Run evaluation (will skip training and load weights)
if Path(model_path).exists():
    results = run_experiment(
        cfg=cfg,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        class_names=cifar10_classes,
        load_weights=True
    )

    experiment_results[results['model_name']] = results

    print("\n" + "="*60)
    print("Model evaluation completed!")
    print("="*60)
else:
    print(f"\nNo saved model found at {model_path}")
    print("Train a model first or check your config parameters.")

In [ ]:
# run multiple experminets

experiments = [
    # Shallow CNN from scratch
    {
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'cnn': {
          'freeze_backbone': False,
          'num_layers': [64, 128],
          'kernel_size': [5, 3],
          'stride': [1, 1],
          'fc_layers': [256],
          'use_dropout': False,
          'use_batch_norm': False
        }
    },

    # Mid CNN from scratch
    {
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'cnn': {
          'freeze_backbone': False,
          'num_layers': [32, 64, 128, 256],
          'kernel_size': [3, 3, 3, 3],
          'stride': [1, 1, 1, 1],
          'fc_layers': [512, 128],
          'dropout_rate': 0.3
        }
    },

    # Deep CNN from scratch
    {
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'cnn': {
          'freeze_backbone': False,
          'num_layers': [32, 64, 64, 128, 128, 256],
          'kernel_size': [3, 3, 3, 3, 3, 3],
          'stride': [1, 1, 1, 1, 1, 1],
          'fc_layers': [512, 256, 128],
          'dropout_rate': 0.4
        }
    },

    # ResNet50
    {
        'model_name': 'ResNet50',
        'mode': 'scratch'
    },

    # Pretrained ResNet18 unfrozen (all layers trainable)
    {'model_name': 'ResNet18', 'mode': 'pretrained', 'freeze_backbone': False},

    # Pretrained ResNet18 frozen (only classifier head trainable)
    {'model_name': 'ResNet18', 'mode': 'pretrained', 'freeze_backbone': True},
]

for exp in experiments:
    cfg.mode = exp['mode']
    cfg.model.model_name = exp['model_name']
    if 'cnn' in exp:
      cfg.model.cnn = exp['cnn']

    print_exp_params(cfg)

    results = run_experiment(
        cfg=cfg,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        class_names=cifar10_classes,
        pretrained=exp['pretrained'],
        freeze_backbone=exp['freeze_backbone']
    )

    exp_name = results['model_name']
    experiment_results[exp_name] = results

# Compare all experiments
compare_experiments(experiment_results, save_path="results/model_comparison.png")